In [1]:
import os 
os.chdir('../')
%pwd

'/home/paladin/Downloads/Bixi-OD-Matrix-Prediction/Bixi-OD-Matrix-Prediction'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareTimeseriesBaseModelConfig:
    root_dir: Path
    base_od_timeseries_model_path: Path
    base_tensor_timeseries_model_path: Path
    trained_od_autoencoder_path: Path
    trained_tensor_autoencoder_path: Path
    params_learning_rate: float
    params_time_lag : int

In [3]:
from timeseriesPredictor.constants import *
from timeseriesPredictor.utils import read_yaml, create_directories

In [4]:
class configurationManeger:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH,                 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.secret = read_yaml(secret_filepath)        
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_prepare_timeseries_base_model_config(self) -> PrepareTimeseriesBaseModelConfig:
        config = self.config.prepare_timeseries_base_model
        
        create_directories([config.root_dir])

        prepare_timeseries_base_model_config = PrepareTimeseriesBaseModelConfig(
            root_dir = config.root_dir,           
            base_od_timeseries_model_path = config.base_od_timeseries_model_path,    
            base_tensor_timeseries_model_path = config.base_tensor_timeseries_model_path,                   
            trained_od_autoencoder_path = self.config.training_autoencoder.trained_od_model_path,
            trained_tensor_autoencoder_path= self.config.training_autoencoder.trained_tensor_model_path,
            params_learning_rate = self.params.LEARNING_RATE_TIMESERIES ,
            params_time_lag = self.params.TIME_LAG                

        )

        return prepare_timeseries_base_model_config

In [5]:
import keras
from timeseriesPredictor.logger import logging
from box import ConfigBox

2023-08-18 15:27:39.824134: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-18 15:27:39.885794: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-18 15:27:39.886633: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 15:27:41.112559: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
class PrepareTimeseriesBaseModel:
    def __init__(self, config: PrepareTimeseriesBaseModelConfig):
        self.config = config
    
    @staticmethod
    def get_encoder_layer_size(autoencoder_path:Path):
        autoencoder = keras.models.load_model(autoencoder_path)
        encoderLayer = autoencoder.get_layer("encoder")              
        return ConfigBox({'input_shape': autoencoder.input_shape[1:],
                         'bottleneck_shape': encoderLayer.output_shape[1:]})
    
    @staticmethod
    def save_model(path:Path, model:keras.Model):
        model.save(path)

    @staticmethod
    def _prepare_full_model(learning_rate, input_shape, reshape):      
        
        # Construct the input layer with no definite frame size.
        inputs = keras.layers.Input(shape= input_shape)

        # We will construct 2 `ConvLSTM2D` layers with batch normalization,
        # followed by `Conv3D` and `Conv2D` layer for the spatiotemporal outputs.
        x = keras.layers.ConvLSTM2D(
            filters=64,
            kernel_size=(3, 3),
            padding="same",
            return_sequences=True,
            recurrent_activation='tanh',
            activation="relu",
            recurrent_dropout=0
        )(inputs)
        x = keras.layers.TimeDistributed(keras.layers.BatchNormalization())(x)
        x = keras.layers.TimeDistributed(keras.layers.Dropout(0.25))(x)
        
        x = keras.layers.ConvLSTM2D(
            filters=32,
            kernel_size=(3, 3),
            padding="same",
            return_sequences=True,
            activation="relu",
            recurrent_activation='tanh',
            recurrent_dropout=0
        )(x)
        x = keras.layers.TimeDistributed(keras.layers.BatchNormalization())(x)
        x = keras.layers.TimeDistributed(keras.layers.Dropout(0.25))(x)        
        
        x = keras.layers.Conv3D(
            filters=16, kernel_size=(3, 3, 3), activation="relu", padding="same"
        )(x)

        x = keras.layers.Conv2D(
            filters=16, kernel_size=(3, 3), activation="relu", padding="same"
        )(x)

        x = keras.layers.Reshape(reshape)(x)
        x = keras.layers.Conv2D(
            filters=1, kernel_size=(3, 3), activation="linear", padding="same"
        )(x)        

        # Next, we will build the complete model and compile it.
        model = keras.models.Model(inputs, x)
        model.compile(
            loss='mean_squared_error', 
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            metrics=['MSE']
            )        
        
        model.summary()
        return model
    
    def update_model(self):
        
        autoencoder_pathes = [self.config.trained_od_autoencoder_path, self.config.trained_tensor_autoencoder_path]
        model_pathes = [self.config.base_od_timeseries_model_path, self.config.base_tensor_timeseries_model_path]
        
        for autoencoder_path, model_path  in zip(autoencoder_pathes, model_pathes):
            row, col, channel =  self.get_encoder_layer_size(autoencoder_path).bottleneck_shape            
            input_shape = [row, col, channel, self.config.params_time_lag]
            row, col, _ =  self.get_encoder_layer_size(autoencoder_path).input_shape
            reshape = [row, col, channel]            

            self.full_model = self._prepare_full_model(        
            learning_rate = self.config.params_learning_rate,
            input_shape = input_shape,
            reshape = reshape            
            )

            self.save_model(path=model_path , model=self.full_model)
            logging.info(f"Timeseries base model saved at {model_path}!")

In [7]:
import sys
from timeseriesPredictor.exception import CustomException

In [8]:
try:
    config = configurationManeger()
    prepare_timeseries_base_model_config = config.get_prepare_timeseries_base_model_config()
    prepare_timeseries_base_model = PrepareTimeseriesBaseModel(config=prepare_timeseries_base_model_config)    
    prepare_timeseries_base_model.update_model()

except Exception as e:
    raise CustomException(e, sys)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16, 16, 3, 7)]    0         
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 16, 16, 3, 64)     163840    
                                                                 
 time_distributed (TimeDist  (None, 16, 16, 3, 64)     256       
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 16, 16, 3, 64)     0         
 stributed)                                                      
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 16, 16, 3, 32)     110720    
                                                                 
 time_distributed_2 (TimeDi  (None, 16, 16, 3, 32)     128   

/home/paladin/Downloads/Bixi-OD-Matrix-Prediction/Bixi-OD-Matrix-Prediction/venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 25, 16, 3, 7)]    0         
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 25, 16, 3, 64)     163840    
                                                                 
 time_distributed_4 (TimeDi  (None, 25, 16, 3, 64)     256       
 stributed)                                                      
                                                                 
 time_distributed_5 (TimeDi  (None, 25, 16, 3, 64)     0         
 stributed)                                                      
                                                                 
 conv_lstm2d_3 (ConvLSTM2D)  (None, 25, 16, 3, 32)     110720    
                                                                 
 time_distributed_6 (TimeDi  (None, 25, 16, 3, 32)     128 